In [5]:
import os
import pandas as pd
import numpy as np

In [6]:
MASK_FEATS = ["id", "label"]
XL_PATH = r"radiomicsFeatures.csv"
OUT_DIR = r"outputs/random"

In [7]:
class RandomFS(object):

    def __init__(self):
        pass

    def __call__(self, feats_df, mask_feats):

        
        features = feats_df.columns[~feats_df.columns.isin(mask_feats)].to_list()
        ranks = np.arange(len(features))+1
        ranks = np.random.permutation(ranks).tolist()

        rank_df = pd.DataFrame({"feature":features, "rank":ranks})

        return rank_df


In [8]:
num_folds = 5

feats_df = pd.read_csv(XL_PATH)

if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)
    
for fold in range(num_folds):

    rank_df = RandomFS()(feats_df, MASK_FEATS)
    rank_df.to_csv(os.path.join(OUT_DIR, f"rank_df{fold}.csv"), index=False)